<h1>IDEA: Un algoritmo de clave privada</h1>

<h2> Cifrado de un mensaje, dados los siguientes datos:</h2>
<li> Mensaje="En un lugar de la Mancha, de cuyo nombre no quiero acordarme"</li>
<li> Clave privada (K) => 128 bits ="MARCOSDIEGOSANTI"

<h2>1. Obtención de los valores binarios del mensaje y la clave </h2>

In [1]:
mensaje_claro="En un lugar de la Mancha, de cuyo nombre no quiero acordarme"
#La clave tiene que usar un alfabeto que contenga menos de 128 bits
clave_privada="MARCOSDIEGOSANTI"
alf="AÁBCDEÉFGHIÍJKLMNÑOÓPQRSTUÚVWXYZ.,;:()¿?¡!-0123456789aábcdeéfghiíjklmnñoópqrstuúvwxyz "

def cadena_a_binario(cadena):
    #Conversion a binario de 8 bits
    cadena_binaria = "".join(format(alf.index(x), '08b') for x in cadena)
    return cadena_binaria

mensaje_binario = cadena_a_binario(mensaje_claro)
clave_binaria = cadena_a_binario(clave_privada)

print(len(mensaje_binario))
print(len(mensaje_claro))


480
60


<h2>2. Se divide el mensaje binario en bloques de 64 bits. Cada bloque se divide a su vez en 4 sub-bloques de 16 bits </h2>


In [2]:
def binario_a_bloques(cadena, tam_bloque):
    lista_bloques = [cadena[x:x+tam_bloque] for x in range(0,len(cadena), tam_bloque)]
    len_ultimo_bloque = len(lista_bloques[-1])
    #Relleno con 0s si no se alcanza el tamaño del bloque
    if(len_ultimo_bloque!=tam_bloque):
        lista_bloques[-1] = lista_bloques[-1]+(tam_bloque-len_ultimo_bloque)*"0"
    return lista_bloques

#Mensaje dividido en bloques de 64 bits, que tendrá la forma [ab,cd]
mensaje_dividido = binario_a_bloques(mensaje_binario, 64)

#Cada sub-bloque lo divido en 4 bloques de 16 bits, guardo el resultado final, que tendrá la forma [[a,b],[c,d]]
mensaje_bloques = []
for bloque in mensaje_dividido:
    mensaje_bloques.append(binario_a_bloques(bloque, 16))

print(mensaje_bloques)



sssssssssssssssssss
[['0000010101000101', '0101010101001110', '0100010101010101', '0100001101001110'], ['0011110100110101', '0100101101010101', '0011100100111010', '0101010101000011'], ['0011010101010101', '0000111100110101', '0100010100111000', '0011111000110101'], ['0010000101010101', '0011100100111010', '0101010100111000', '0100111001010011'], ['0100011101010101', '0100010101000111', '0100010000110111', '0100101100111010'], ['0101010101000101', '0100011101010101', '0100101001001110', '0011111100111010'], ['0100101101000111', '0101010100110101', '0011100001000111', '0100101100111001'], ['0011010101001011', '0100010000111010', '0000000000000000', '0000000000000000']]


<h2>3. Se divide la clave en bloques de 16 bits. Quedará dividida en 8</h2>

In [3]:
clave_dividida = binario_a_bloques(clave_binaria, 16)
print(clave_dividida)

['0000111100000000', '0001011000000011', '0001001000010111', '0000010000001010', '0000010100001000', '0001001000010111', '0000000000010000', '0001100000001010']
